In [14]:
from threading import Thread
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer

from trl import (
    ModelConfig,
    RewardConfig,
    RewardTrainer,
    ScriptArguments,
    get_kbit_device_map,
    get_peft_config,
    get_quantization_config,
    setup_chat_format,
)

from transformers import DataCollatorWithPadding
from typing import List, Dict, Any
from transformers import PreTrainedTokenizerBase

In [15]:
from datasets import load_dataset
ds = load_dataset("Hankbeasley/polycoder")






In [16]:
def preprocess_function(examples):
    parts = examples['accept'].split('<think>')
    if (len(parts) < 2 or len(parts) > 2):
        return None
    assistantparts = examples['accept'].split("<｜Assistant｜>")
    if (len(assistantparts) > 2):
        return None
    examples['prompt'] = parts[0]
    examples['chosen'] = parts[1]
    partsrejected = examples['reject'].split('<think>')
    examples['rejected'] = "<think>" + partsrejected[1]
    return examples

ds = ds['train'].map(preprocess_function)
print(ds)
formated = ds.remove_columns(['accept', 'reject', 'testname'])


Dataset({
    features: ['testname', 'accept', 'reject', 'prompt', 'chosen', 'rejected'],
    num_rows: 8938
})


In [ ]:

#print(formated[0]['prompt'])
#print(formated)
parts = ds['train'][0]['accept'].split('<think>')
prompt = ds['train'][0]['accept'].split('<think>')[0]
#print(parts[1])

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
tokenized = tokenizer(formated[0]['rejected'], add_special_tokens=True)
#print(tokenized["input_ids"])
#print(tokenizer.convert_ids_to_tokens(tokenized["input_ids"]))
#print(tokenizer.eos_token)
